# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse    Shipments  \
0  Feb  1 2023 12:17PM  256397        19   GP02022023   
1  Feb  1 2023 12:15PM  256395        10   0086345837   
2  Feb  1 2023 12:15PM  256395        10   0086345864   
3  Feb  1 2023 12:15PM  256395        10   0086345871   
4  Feb  1 2023 12:15PM  256395        10   0086345872   
5  Feb  1 2023 12:15PM  256395        10   0086345873   
6  Feb  1 2023 11:26AM  256393        10   9072130-BO   
7  Feb  1 2023 11:26AM  256393        10  9072141-BO1   
8  Feb  1 2023 11:26AM  256393        10  9072142-BO1   
9  Feb  1 2023 11:26AM  256393        10  9072143-BO1   

                         Customer ShipmentStatus  
0  Granules Pharmaceuticals, Inc.       Released  
1               ISDIN Corporation       Released  
2               ISDIN Corporation       Released  
3               ISDIN Corporation       Released  
4               ISDIN Corporation       Released  
5               ISDIN Corporation       Released  
6                Eywa Pharma Inc.       Released  
7                Eywa Pharma Inc.       Released  
8                Eywa Pharma Inc.      Executing  
9                Eywa Pharma Inc.      Executing

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
26  256392       Released          1
27  256393      Executing          2
28  256393       Released         11
29  256395       Released          5
30  256397       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
256391                NaN        NaN       1.0
256392                NaN        NaN       1.0
256393                NaN        2.0      11.0
256395                NaN        NaN       5.0
256397                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
256299                0.0        0.0       1.0
256319               19.0        4.0       3.0
256340                0.0        0.0       1.0
256350                0.0        0.0       1.0
256352                0.0       12.0       8.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
256299                  0          0         1
256319                 19          4         3
256340                  0          0         1
256350                  0          0         1
256352                  0         12         8

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               256299          0          0         1
1               256319         19          4         3
2               256340          0          0         1
3               256350          0          0         1
4               256352          0         12         8

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               256299                            1
1               256319        19         4        3
2               256340                            1
3               256350                            1
4               256352                  12        8

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                        Customer
0   Feb  1 2023 12:17PM  256397        19  Granules Pharmaceuticals, Inc.
1   Feb  1 2023 12:15PM  256395        10               ISDIN Corporation
6   Feb  1 2023 11:26AM  256393        10                Eywa Pharma Inc.
19  Feb  1 2023 11:15AM  256392        21               NBTY Global, Inc.
20  Feb  1 2023 11:12AM  256391        21               NBTY Global, Inc.
21  Feb  1 2023 11:07AM  256390        21               NBTY Global, Inc.
22  Feb  1 2023 10:38AM  256387        16  Sartorius Bioprocess Solutions
23  Feb  1 2023 10:34AM  256386        10               ISDIN Corporation
28  Feb  1 2023 10:06AM  256383        10                  ClearSpec, LLC
30  Feb  1 2023  9:43AM  256381        10                  ClearSpec, LLC

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                        Customer  \
0  Feb  1 2023 12:17PM  256397        19  Granules Pharmaceuticals, Inc.   
1  Feb  1 2023 12:15PM  256395        10               ISDIN Corporation   
2  Feb  1 2023 11:26AM  256393        10                Eywa Pharma Inc.   
3  Feb  1 2023 11:15AM  256392        21               NBTY Global, Inc.   
4  Feb  1 2023 11:12AM  256391        21               NBTY Global, Inc.   
5  Feb  1 2023 11:07AM  256390        21               NBTY Global, Inc.   
6  Feb  1 2023 10:38AM  256387        16  Sartorius Bioprocess Solutions   
7  Feb  1 2023 10:34AM  256386        10               ISDIN Corporation   
8  Feb  1 2023 10:06AM  256383        10                  ClearSpec, LLC   
9  Feb  1 2023  9:43AM  256381        10                  ClearSpec, LLC   

  Completed Executing Released  
0                            1  
1                            5  
2                   2       11  
3                            1  
4                            1  
5                            1  
6                            1  
7                   5           
8                            2  
9                   1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                        Customer  \
0  Feb  1 2023 12:17PM  256397        19  Granules Pharmaceuticals, Inc.   
1  Feb  1 2023 12:15PM  256395        10               ISDIN Corporation   
2  Feb  1 2023 11:26AM  256393        10                Eywa Pharma Inc.   
3  Feb  1 2023 11:15AM  256392        21               NBTY Global, Inc.   
4  Feb  1 2023 11:12AM  256391        21               NBTY Global, Inc.   
5  Feb  1 2023 11:07AM  256390        21               NBTY Global, Inc.   
6  Feb  1 2023 10:38AM  256387        16  Sartorius Bioprocess Solutions   
7  Feb  1 2023 10:34AM  256386        10               ISDIN Corporation   
8  Feb  1 2023 10:06AM  256383        10                  ClearSpec, LLC   
9  Feb  1 2023  9:43AM  256381        10                  ClearSpec, LLC   

  Released Executing Completed  
0        1                      
1        5                      
2       11         2            
3        1                      
4        1                      
5        1                      
6        1                      
7                  5            
8        2                      
9                  1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Feb  1 2023 12:17PM  256397        19  Granules Pharmaceuticals, Inc.   
1  Feb  1 2023 12:15PM  256395        10               ISDIN Corporation   
2  Feb  1 2023 11:26AM  256393        10                Eywa Pharma Inc.   
3  Feb  1 2023 11:15AM  256392        21               NBTY Global, Inc.   
4  Feb  1 2023 11:12AM  256391        21               NBTY Global, Inc.   

  Released Executing Completed  
0        1                      
1        5                      
2       11         2            
3        1                      
4        1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Feb  1 2023 12:17PM  256397        19  Granules Pharmaceuticals, Inc.   
1  Feb  1 2023 12:15PM  256395        10               ISDIN Corporation   
2  Feb  1 2023 11:26AM  256393        10                Eywa Pharma Inc.   
3  Feb  1 2023 11:15AM  256392        21               NBTY Global, Inc.   
4  Feb  1 2023 11:12AM  256391        21               NBTY Global, Inc.   

   Released  Executing  Completed  
0       1.0        NaN        NaN  
1       5.0        NaN        NaN  
2      11.0        2.0        NaN  
3       1.0        NaN        NaN  
4       1.0        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Feb  1 2023 12:17PM  256397        19  Granules Pharmaceuticals, Inc.   
1  Feb  1 2023 12:15PM  256395        10               ISDIN Corporation   
2  Feb  1 2023 11:26AM  256393        10                Eywa Pharma Inc.   
3  Feb  1 2023 11:15AM  256392        21               NBTY Global, Inc.   
4  Feb  1 2023 11:12AM  256391        21               NBTY Global, Inc.   

   Released  Executing  Completed  
0       1.0        0.0        0.0  
1       5.0        0.0        0.0  
2      11.0        2.0        0.0  
3       1.0        0.0        0.0  
4       1.0        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         2051031      64.0       16.0        0.0
15          769082      37.0       29.0        4.0
16          769026       3.0        0.0        0.0
19         1538149      14.0       17.0       19.0
21          769173       3.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  2051031      64.0       16.0        0.0
1        15   769082      37.0       29.0        4.0
2        16   769026       3.0        0.0        0.0
3        19  1538149      14.0       17.0       19.0
4        21   769173       3.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      64.0       16.0        0.0
1        15      37.0       29.0        4.0
2        16       3.0        0.0        0.0
3        19      14.0       17.0       19.0
4        21       3.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   64.0
1        15  Released   37.0
2        16  Released    3.0
3        19  Released   14.0
4        21  Released    3.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    15   16    19   21
Status                               
Completed   0.0   4.0  0.0  19.0  0.0
Executing  16.0  29.0  0.0  17.0  0.0
Released   64.0  37.0  3.0  14.0  3.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    15   16    19   21
0          Completed   0.0   4.0  0.0  19.0  0.0
1          Executing  16.0  29.0  0.0  17.0  0.0
2           Released  64.0  37.0  3.0  14.0  3.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    15   16    19   21
0  Completed   0.0   4.0  0.0  19.0  0.0
1  Executing  16.0  29.0  0.0  17.0  0.0
2   Released  64.0  37.0  3.0  14.0  3.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()